In [ ]:
!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
!mkdir speechcommands
!tar -xf speech_commands_v0.01.tar.gz -C /content/speechcommands

In [1]:
import librosa
import glob
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import Audio
from librosa.display import specshow

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# generamos lista con las rutas y nombres de archivo de los digitos
numeros = []
digitnames = ('zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine')
for i in digitnames:
    path = 'speechcommands/' + i + '/*.wav'
    numeros.append(glob.glob(path))

In [ ]:
# ver que onda

x,sr = librosa.core.load(numeros[2][4],sr=None)
plt.plot(x)
Audio(x,rate=sr)

In [ ]:
def calculate_features(filename, n_mfcc=12, delta=True, deltadelta=True, energy=True, summary_fn = [np.mean, np.std], summary_names=['mean', 'std']):
  #Abro el archivo:
  x, sr = librosa.core.load(filename,sr=None)
  
  #Calculo MFCCs
  features = librosa.feature.mfcc(x,sr=sr,n_mfcc=n_mfcc)

  #Calculo energia:
  if energy:
    energy = librosa.feature.rmse(x)
    features = np.concatenate([features,energy])
    feat_names = feat_names + ['energy']
  #Aplico media y desvio estandar por defecto
  summary_features = np.concatenate([fn(features,axis=1) for fn in summary_fn])
  ##feat_names = ['{}_{}'.format(name_i,summ_i) for summ_i in summary_names for name_i in feat_names]
  
  #Lo mismo con los delta
  if delta:
    deltafeatures = np.diff(features)
    summary_features = np.concatenate([summary_features,np.concatenate([fn(deltafeatures,axis=1) for fn in summary_fn])])
    ##d_names = ['d{}'.format(name) for name in feat_names]
  else:
    d_names = []

  #Y con los delta de segundo orden
  if deltadelta:
    deltadeltafeatures = np.diff(features,n=2)
    summary_features = np.concatenate([summary_features,np.concatenate([fn(deltadeltafeatures,axis=1) for fn in summary_fn])]) 
    ##dd_names = ['dd{}'.format(name) for name in feat_names]
  else:
    dd_names = []

  ##feat_names = feat_names + d_names + dd_names + ['digit', 'file']
  summary_features = np.append(summary_features, [filename.split('/')[1], filename])

  return summary_features##, feat_names 

def name_features(filename = '', n_mfcc=12, delta=True, deltadelta=True, energy=True, summary_fn = [np.mean, np.std], summary_names=['mean', 'std']):
    feat_names = ['mfcc_{}'.format(i) for i in range(n_mfcc)]
    feat_names = ['{}_{}'.format(name_i,summ_i) for summ_i in summary_names for name_i in feat_names]
    d_names = ['d{}'.format(name) for name in feat_names]
    dd_names = ['dd{}'.format(name) for name in feat_names]

    feat_names = feat_names + d_names + dd_names + ['digit', 'file']

    return feat_names

In [ ]:
# guardamos la lista de los conjuntos de test y validación
# solo para archivos que referencian a dígitos
test_idxs = ['speechcommands/' + e for e in open('speechcommands/testing_list.txt','r').read().splitlines() if e[:e.find('/')] in digitnames]
valid_idxs = ['speechcommands/' + e for e in open('speechcommands/validation_list.txt','r').read().splitlines() if e[:e.find('/')] in digitnames]

train_idxs = [e for n in range(10) for e in numeros[n] if (e not in test_idxs) and (e not in valid_idxs)]

print(len(train_idxs), len(test_idxs) ,len(valid_idxs))

In [ ]:
# como siempre calculamos las mismas fatures, sus nombres los guardo una sola vez
feat_test = []
feat_valid = []
feat_train = []

feat_test = [calculate_features(x) for x in test_idxs]
feat_valid = [calculate_features(x) for x in valid_idxs]
feat_train = [calculate_features(x) for x in train_idxs]
feat_names = name_features()

#for n in range(2):
#    for i in range(len(numeros[n])//100):
#        feat_, names_ = calculate_features(numeros[n][i])
#        feat.append(feat_)
#        names = names_

print(feat_names)